In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report



In [2]:
def connect_to_db():
    return mysql.connector.connect(
       host="localhost",
       user="root",
       password="",
       database="leave_data"
    )

In [8]:
def fetch_leave_data(email):
    try:
        conn=connect_to_db()
        cursor=conn.cursor(dictionary=True)
        query="""
        SELECT l.id, lb.empname, lb.cl, lb.sl, lb.co, lb.empemail, l.hrremark, l.mgrremark, l.aprremark, l.from, l.to, l.desg
        FROM leavebalance lb
        JOIN leaves l ON lb.empemail = l.empemail
        WHERE lb.empemail = %s
        """
        cursor.execute(query,(email,))
        result=cursor.fetchall()
        conn.close()
        return pd.DataFrame(result) if result else None
    except mysql.connector.Error as err:
        print(f"Error:{err}")
        return None

In [9]:
#leave eligibility

def can_apply_leave(data):
    if data is not None and not data.empty:
        # Convert columns to numeric, handling non-numeric values
        data["cl"] = pd.to_numeric(data["cl"], errors='coerce').fillna(0)
        data["sl"] = pd.to_numeric(data["sl"], errors='coerce').fillna(0)
        data["co"] = pd.to_numeric(data["co"], errors='coerce').fillna(0)
        
        # Calculate total leave balance (lb = cl + sl + co)
        data["lb"] = data["cl"] + data["sl"] + data["co"]
        
        # Check if leave can be applied (i.e., if total balance > 0)
        data["can_apply_leave"] = data["lb"] > 0
        return data
    else:
        return "not having enough leave balance"


In [11]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Load the dataset for leave classification
df = pd.read_csv(r"c:\HarshaPriya\ML\type_of_leave.csv")  # Adjust path to your dataset

# Preprocessing function
def preprocess(text):
    text = text.lower()  # Lowercasing
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    words = text.split()  # Tokenization
    unwanted_keywords = ["feeling"]
    words = [word for word in words if word not in unwanted_keywords]
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

# Apply preprocessing to the 'text' column in the dataset
df['processed_text'] = df['text'].apply(preprocess)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\it\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:

# Initialize TF-IDF vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)


In [13]:

# Transform the 'processed_text' into TF-IDF features
X = vectorizer.fit_transform(df['processed_text'])

# Target labels (y) would be from the 'label' column
y = df['label']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Logistic Regression and SVM models
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)
svm_model = SVC(kernel='linear', random_state=42)

# Train the models
log_reg_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)



SVC(kernel='linear', random_state=42)

In [ ]:
from datetime import date
# Function to predict whether a given text is a sick leave or not
def predict_sick_leave(text, model, vectorizer):
    # Preprocess the input text
    processed_text = preprocess(text)
    # Convert the text to the feature vector
    text_vector = vectorizer.transform([processed_text])
    # Predict using the trained model
    prediction = model.predict(text_vector)
    # Return result
    if prediction[0] == 1:
        return "Sick Leave"
    else:
        return "Not Sick Leave"

# Form submission callback
def on_submit(button):
    with output:
        clear_output()  # Clear previous outputs
        email = email_input.value
        data = fetch_leave_data(email)
        from_date = from_date_input.value
        to_date = to_date_input.value
        selected_leave_type = leave_type_input.value
        leave_reason = reason_input.value
        leave_status = predict_sick_leave(leave_reason, log_reg_model, vectorizer)  # Assume model and vectorizer are available

        if not from_date or not to_date:
            print("Please select both 'From Date' and 'To Date'.")
            return

        elif from_date < today or to_date < today:
            print("Dates must be today or in the future. Please select valid dates.")
            return

        elif from_date > to_date:
            print("'From Date' cannot be later than 'To Date'. Please correct the dates.")
            return

        delta = (to_date - from_date).days + 1  

        # if delta > 6:
        #     print("You cannot apply for leave more than 6 days.")
        if delta > 3:
            print("Needs HR Review becuase requesting leave days is exceeding more than 3 days")
        else:
            if data is not None and not data.empty:
            # Check leave eligibility (whether user has enough balance)
                data = can_apply_leave(data)
                
                
                if data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Casual Leave" and leave_status != "Sick Leave":
                    if today < from_date:  # From Date must be greater than or equal to today
                        if delta <= data["lb"].iloc[0]:
                            print("Leave Granted")
                        else:
                            print("You do not have enough leave balance to apply for leave more than your leave balance.")
                    else:
                        print("The 'From Date' should be greater than or equal to today's date.")
                     
                elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Sick Leave" and leave_status == "Sick Leave":
                    print("You do not have enough leave balance so it needs hr review")  
                    
                elif data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Sick Leave" and leave_status == "Sick Leave":
                    print("Leave Granted")  

                
                elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Casual Leave" and leave_status != "Sick Leave":
                    print("You do not have enough leave balance to apply for leave so it needs HR review")
                    print("There will be LOP")
                    
                
                elif data["can_apply_leave"].iloc[0] == False and selected_leave_type == "Casual Leave" and leave_status == "Sick Leave":
                    print("Leave type isn't matching with reason. Repeated mismatches may result in further action.")
                    
                elif data["can_apply_leave"].iloc[0] == True and selected_leave_type == "Casual Leave" and leave_status == "Sick Leave":
                    print("Leave type isn't matching with reason.")
                    
                elif selected_leave_type == "Sick Leave" and leave_status != "Sick Leave":
                    print("Leave type isn't matching with reason. Repeated mismatches may result in further action.")
                    return 
                    
                else:
                    print("You do not have enough leave balance to apply for leave.")
            else:
                print("No leave data found for the provided email.")

# Create widgets for the form
leave_type_input = widgets.Dropdown(
    options=['Casual Leave', 'Sick Leave', 'On Duty'],
    description='Leave Type:',
    disabled=False
)
email_input = widgets.Text(description="Email:")

# Create date picker widgets for "From" and "To" inputs with min date set to today
today = date.today()
from_date_input = widgets.DatePicker(
    description="From Date",
    disabled=False,
    min=today  # Prevent selection of dates before today
)
to_date_input = widgets.DatePicker(
    description="To Date",
    disabled=False,
    min=today  # Prevent selection of dates before today
)

reason_input = widgets.Text(description="Reason:")

submit_button = widgets.Button(description="Fetch Data:")
output = widgets.Output()

# Attach event listener to the button
submit_button.on_click(on_submit)

# Display the form
display(widgets.VBox([ 
    leave_type_input, 
    email_input, 
    from_date_input, 
    to_date_input, 
    reason_input, 
    submit_button, 
    output 
]))